In [1]:
import os
import pickle
import random
from tqdm import tqdm, tqdm_notebook
import numpy as np
from sklearn.model_selection import train_test_split
from keras import optimizers, losses
from ph_likes.util import settings
from ph_likes.util.ml.models import embedding_model
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED)

/home/can/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA = dict()
DATA['generate_training_data'] = False

# Load Data

In [3]:
# CF = np.load(os.path.join(settings.LOCAL_MODELS_PATH, 'CF.npy'))
# print(CF.mean())
# print(CF.sum())
# print(CF.shape)

In [4]:
# post_ids
post_ids2index = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'post_ids2index.pkl'), 'rb'))
index2post_ids = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'index2post_ids.pkl'), 'rb'))
#
user_ids2index = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'user_ids2index.pkl'), 'rb'))
index2user_ids = pickle.load(open(os.path.join(settings.LOCAL_MODELS_PATH, 'index2user_ids.pkl'), 'rb'))

In [5]:
DATA['N_falses_samples_for_each_user'] = 2
DATA['N_USERS'], DATA['N_POSTS'] = len(post_ids2index), len(user_ids2index)

# Create Training Data

In [6]:
def sample_max(arr, n):
    if len(arr) > n:
        return np.random.choice(arr, size=n, replace=False)
    else:
        return arr

In [7]:
if DATA['generate_training_data']:
    # generate false indices
    # - Sample n negatives from each user
    DATA['false_indices'] = list()
    for i_user in tqdm_notebook(range(CF.shape[0])):
        indices = np.where(np.invert(CF[i_user]))[0]
        if len(indices) > 0:
            indices = sample_max(indices, DATA['N_falses_samples_for_each_user'])
            for indice in indices:
                DATA['false_indices'].append((i_user, indice))
        else:
            continue
    DATA['false_indices'] = np.array(DATA['false_indices'])
    DATA['false_indices'] = [DATA['false_indices'][:, 0], DATA['false_indices'][:, 1]]
    
    # generate true indices
    DATA['true_indices'] = np.where(CF)
    
    CF = None  # No longer neccessary
    
    # merge
    DATA['X'] = [np.array(np.concatenate([DATA['true_indices'][0], DATA['false_indices'][0]]), dtype=np.uint8), 
                 np.array(np.concatenate([DATA['true_indices'][1], DATA['false_indices'][1]]), dtype=np.uint8)]
    DATA['y'] = np.array([1] * len(DATA['true_indices'][0]) + [0] * len(DATA['false_indices'][0]), dtype=np.uint8)
    
    DATA['false_indices'] = None
    DATA['true_indices'] = None
    
    # saving
    np.save(os.path.join(settings.LOCAL_DATA_PATH, 'X.npy'), DATA['X'])
    np.save(os.path.join(settings.LOCAL_DATA_PATH, 'y.npy'), DATA['y'])
else:
    DATA['X'] = np.load(os.path.join(settings.LOCAL_DATA_PATH, 'X.npy'))
    DATA['y'] = np.load(os.path.join(settings.LOCAL_DATA_PATH, 'y.npy'))

In [8]:
print('X[0].shape:', DATA['X'][0].shape)
print('X[1].shape:', DATA['X'][1].shape)
print('y.shape:', DATA['y'].shape)

X[0].shape: (12122455,)
X[1].shape: (12122455,)
y.shape: (12122455,)


# Model

In [9]:
model = embedding_model.get_embedding_model(n_users=DATA['N_USERS'], n_posts=DATA['N_POSTS'])

In [10]:
optimizer = optimizers.Adam()
loss = losses.binary_crossentropy
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Training

In [11]:
X = [np.expand_dims(DATA['X'][0], axis=1), np.expand_dims(DATA['X'][1], axis=1)]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, DATA['y'])

ValueError: Found input variables with inconsistent numbers of samples: [2, 12122455]

In [21]:
model.fit(X, DATA['y'], batch_size=1024, epochs=50, class_weight={0:6, 1:1}, validation_data=(X_test, y_test))

Epoch 1/50
12122455/12122455 [==============================] - 86s 7us/step - loss: 1.1212 - acc: 0.7556
Epoch 2/50
12122455/12122455 [==============================] - 85s 7us/step - loss: 1.1198 - acc: 0.7278
Epoch 3/50
12122455/12122455 [==============================] - 86s 7us/step - loss: 1.1194 - acc: 0.7255
Epoch 4/50
12122455/12122455 [==============================] - 86s 7us/step - loss: 1.1192 - acc: 0.7219
Epoch 5/50
12122455/12122455 [==============================] - 86s 7us/step - loss: 1.1190 - acc: 0.7201
Epoch 6/50
 2070528/12122455 [====>.........................] - ETA: 1:12 - loss: 1.1188 - acc: 0.7176

KeyboardInterrupt: 